In [7]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
pd.options.mode.chained_assignment = None
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

In [8]:
# Load Saved Results

data = pd.read_csv('gen_saved_pred.csv')

#data = pd.read_csv('../../../../Desktop/NewResults/gen_saved_pred.csv')
#data = data[data['num_train']==1]

#data = pd.read_csv('../../../../Desktop/NewResults/ind_saved_pred.csv')
#data = data[(data['test_perc']>=0.47) & (data['test_perc']<=0.53)]

In [9]:
data.head()

,num_train,participant,session_num,timestamp,actual,pred,score,timestamp_old,ideal_25,ideal_50,ideal_75,disengagement_index,sequence_id,sequence_length
0,6,7.0,5.0,42.5,0.0,0.0,0.326539,-1.562899,False,False,False,0.5,271,1.0
1,6,7.0,5.0,43.0,0.0,0.0,0.350756,-1.561677,False,False,False,1.0,271,1.0
2,6,7.0,5.0,43.5,1.0,1.0,0.810333,-1.560456,False,False,False,NaN,272,NaN
3,6,7.0,5.0,44.0,1.0,1.0,0.976211,-1.559234,False,False,False,NaN,272,NaN
4,6,7.0,5.0,44.5,1.0,1.0,0.988275,-1.558013,False,False,False,NaN,272,NaN


In [ ]:
windows =[2,4,6,8,10,12,14,16,18,20]

track_scores = {}
for w in windows:
    track_scores[w] = []
    
track_preds = {}
for w in windows:
    track_preds[w] = []

checkpoint = int(len(data)/10)

exp_id = 0
exp_ids = []

last_participant = -1
last_session = -1
last_num_train = -1

for i,r in data.iterrows():
    if i%checkpoint == 0:
        print(i)
    
    if i>0 and (last_participant != r['participant'] 
                or last_session != r['session_num']
                or last_num_train != r['num_train']):
        exp_id += 1
        last_participant = r['participant']
        last_session = r['session_num']
        last_num_train = r['num_train']
        
    exp_ids.append(exp_id)

    for w in windows:        
        if i+1 < w:
            track_scores[w].append(np.nan)
            track_preds[w].append(np.nan)
            continue

        if data.loc[i-w+1, 'participant'] != r['participant']:
            track_scores[w].append(np.nan)
            track_preds[w].append(np.nan)
            continue
        if data.loc[i-w+1, 'session_num'] != r['session_num']:
            track_scores[w].append(np.nan)
            track_preds[w].append(np.nan)
            continue

        score_signal = (data.loc[i-w+1:i,'score'].sum())
        track_scores[w].append(score_signal/w)
        
        pred_signal = (data.loc[i-w+1:i,'pred'].sum())
        track_pred[w].append(pred_signal/w)

In [ ]:
data['exp_id'] = exp_ids

score_signals = []
pred_signals = []

for w in windows:
    col = 'score_signal_'+str(w)
    score_signals.append(col)
    data[col] = track_scores[col]
    
    col = 'pred_signal_'+str(w)
    pred_signals.append(col)
    data[col] = track_pred[col]

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data.to_csv()

In [ ]:
pd.to_csv('gen_saved_pred2.csv')